# Decision Trees

A nonlinear, human-readable classification or regression tool.

## Cereal Data

Pulled some data off of Kaggle: <https://www.kaggle.com/crawford/80-cereals>

Let's learn about Cereal!

In [1]:
import pandas as pd

input_file = "data/cereal.csv"

Pandas is a great tool for manipulating data and provide a great datastructure
called a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)

<https://pandas.pydata.org>

In [2]:
# easily pull in csv data with pandas dataframes
dataset = pd.read_csv(input_file)

# We can explore the data a bit with Pandas helper methods
print(dataset.head())
print(dataset.describe())

                        name mfr type  calories  protein  fat  sodium  fiber  \
0                  100% Bran   N    C        70        4    1     130   10.0   
1          100% Natural Bran   Q    C       120        3    5      15    2.0   
2                   All-Bran   K    C        70        4    1     260    9.0   
3  All-Bran with Extra Fiber   K    C        50        4    0     140   14.0   
4             Almond Delight   R    C       110        2    2     200    1.0   

   carbo  sugars  potass  vitamins  shelf  weight  cups     rating  
0    5.0       6     280        25      3     1.0  0.33  68.402973  
1    8.0       8     135         0      3     1.0  1.00  33.983679  
2    7.0       5     320        25      3     1.0  0.33  59.425505  
3    8.0       0     330        25      3     1.0  0.50  93.704912  
4   14.0       8      -1        25      3     1.0  0.75  34.384843  
         calories    protein        fat      sodium      fiber      carbo  \
count   77.000000  77.000000

Many algorithms can't work on categorical data directly. The data needs to be
transformed into a numerical representation of the categorical or string data.

In [3]:
from sklearn.preprocessing import LabelBinarizer

# remove these string or categorical features...
X = dataset.drop(['name', 'type', 'rating'], axis=1)

# create the encoder and 'fit' it
mfr_encoder = LabelBinarizer()
mfr_encoder.fit(X['mfr'])
transformed_feature = mfr_encoder.transform(X['mfr'])
# convert to dataframe for ease of manipulating it
tr_df = pd.DataFrame(transformed_feature)

# essentially replace the 'mfr' feature with the newly encoded version of it
X = pd.concat([X, tr_df], axis=1).drop(['mfr'], axis=1)

print(X.head())
print(X.dtypes)

   calories  protein  fat  sodium  fiber  carbo  sugars  potass  vitamins  \
0        70        4    1     130   10.0    5.0       6     280        25   
1       120        3    5      15    2.0    8.0       8     135         0   
2        70        4    1     260    9.0    7.0       5     320        25   
3        50        4    0     140   14.0    8.0       0     330        25   
4       110        2    2     200    1.0   14.0       8      -1        25   

   shelf  weight  cups  0  1  2  3  4  5  6  
0      3     1.0  0.33  0  0  0  1  0  0  0  
1      3     1.0  1.00  0  0  0  0  0  1  0  
2      3     1.0  0.33  0  0  1  0  0  0  0  
3      3     1.0  0.50  0  0  1  0  0  0  0  
4      3     1.0  0.75  0  0  0  0  0  0  1  
calories      int64
protein       int64
fat           int64
sodium        int64
fiber       float64
carbo       float64
sugars        int64
potass        int64
vitamins      int64
shelf         int64
weight      float64
cups        float64
0             int64
1

Let's create the labels next, based off the Consumer Reports ratings...followed
up by creating the train/test split of the data.

In [4]:
from sklearn.model_selection import train_test_split

Y = dataset['rating']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## SKLearn's Decision Tree

In [5]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
dt.fit(X_train, Y_train)
y_pred = dt.predict(X_test)

pred_df = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})

## Visualizing Decision Trees

We can invert the LabelBinarizer, but that would produce the original category
labels which still isn't that readable. So I put the cereal names into a list.

In [6]:
from sklearn import tree

mfr_names = ['American Home Food Products',
                 'General Mills',
                 'Kelloggs',
                 'Nabisco',
                 'Post',
                 'Quaker Oats',
                 'Ralston Purina']
feature_names = list(X.columns.values)[:-7] + mfr_names
text_representation = tree.export_text(dt, feature_names=feature_names)
print(text_representation)

|--- sugars <= 6.50
|   |--- Nabisco <= 0.50
|   |   |--- calories <= 95.00
|   |   |   |--- shelf <= 2.00
|   |   |   |   |--- value: [49.12]
|   |   |   |--- shelf >  2.00
|   |   |   |   |--- Post <= 0.50
|   |   |   |   |   |--- Kelloggs <= 0.50
|   |   |   |   |   |   |--- value: [63.01]
|   |   |   |   |   |--- Kelloggs >  0.50
|   |   |   |   |   |   |--- cups <= 0.67
|   |   |   |   |   |   |   |--- value: [59.43]
|   |   |   |   |   |   |--- cups >  0.67
|   |   |   |   |   |   |   |--- value: [59.64]
|   |   |   |   |--- Post >  0.50
|   |   |   |   |   |--- value: [53.31]
|   |   |--- calories >  95.00
|   |   |   |--- protein <= 2.50
|   |   |   |   |--- fat <= 0.50
|   |   |   |   |   |--- carbo <= 21.50
|   |   |   |   |   |   |--- Ralston Purina <= 0.50
|   |   |   |   |   |   |   |--- calories <= 105.00
|   |   |   |   |   |   |   |   |--- value: [45.86]
|   |   |   |   |   |   |   |--- calories >  105.00
|   |   |   |   |   |   |   |   |--- value: [46.90]
|   |   |   |

### matplotlib images

We can also produce images from this library.

In [7]:
from matplotlib import pyplot as plt

fig = plt.figure(figsize=(100, 100))
fig.set_dpi(100.0)
_ = tree.plot_tree(dt,
                   feature_names=feature_names,
                   filled=True)
fig.savefig("decision_tree.png")